---
title: "Graded Problem Set: Investment and Portfolio Management"
subtitle: "Imperial College London - Business School"
author: 
  - name: "Rodolphe Lajugie"
date: "2025-10-31"
format: pdf
fontsize: 12pt
geometry: margin=1in
titlepage: true
titlepage-geometry: "top=2cm, bottom=2cm, left=2cm, right=2cm"
---
\newpage
\tableofcontents
\listoffigures
\listoftables
\newpage

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

Let's begin the analysis by looking at the structure of the dataset.

In [2]:
stock_prices = pd.read_excel('Problem set data.xls', sheet_name='Stock Prices')  

In [3]:
stock_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          216 non-null    object 
 1   Ticker        216 non-null    object 
 2   Company Name  216 non-null    object 
 3   Close Price   216 non-null    float64
 4   High Price    216 non-null    float64
 5   Low Price     216 non-null    float64
dtypes: float64(3), object(3)
memory usage: 10.3+ KB


In [4]:
# | label: stock_prices_info__str_dataset
# | tbl-cap: "Quick information about the string columns in the stock_prices table."
stock_prices.describe(include='object')

,Date,Ticker,Company Name
count,216,216,216
unique,72,3,3
top,2019-01,AAPL,APPLE INC
freq,3,72,72


In [5]:
# | label: stock_prices_info_int_dataset
# | tbl-cap: "Quick information about the numerical columns in the stock_prices table."
stock_prices.describe(include='number')

,Close Price,High Price,Low Price
count,216.000000,216.000000,216.000000
mean,608.299352,645.881990,563.092460
std,1032.094012,1100.958804,959.739325
min,6.070000,7.000000,5.480000
25%,102.882500,107.327500,95.775000
50%,160.395000,166.989950,144.275000
75%,230.000000,237.295000,212.937500
max,3507.070000,3773.078200,3306.980000


In [6]:
mkt_index = pd.read_excel('Problem set data.xls', sheet_name='Market Index')

In [7]:
mkt_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        72 non-null     object 
 1   Level of the S&P 500 Index  72 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.3+ KB


In [8]:
# | label: mkt_index_info__str_dataset
# | tbl-cap: "Quick information about the string columns in the mkt_index table."
mkt_index.describe(include='object')

,Date
count,72
unique,72
top,2019-01
freq,1


In [9]:
# | label: mkt_index_info_int_dataset
# | tbl-cap: "Quick information about the numerical columns in the mkt_index table."
mkt_index.describe(include='number')

,Level of the S&P 500 Index
count,72.000000
mean,4049.033194
std,873.791990
min,2584.590000
25%,3260.165000
50%,4119.800000
75%,4539.557500
max,6032.380000


In [10]:
rf_rate = pd.read_excel('Problem set data.xls', sheet_name='Risk Free Rate')

In [11]:
rf_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         72 non-null     object 
 1   Return on the T bill (in %)  72 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.3+ KB


In [12]:
# | label: rf_rate_info__str_dataset
# | tbl-cap: "Quick information about the string columns in the rf_rate table."
rf_rate.describe(include='object')

,Date
count,72
unique,72
top,2019-01
freq,1


In [13]:
# | label: rf_rate_info_int_dataset
# | tbl-cap: "Quick information about the numerical columns in the rf_rate table."
rf_rate.describe(include='number')

,Return on the T bill (in %)
count,72.000000
mean,2.431667
std,2.206670
min,0.010000
25%,0.090000
50%,2.110000
75%,4.762500
max,5.570000


# A. Coveriance Matrix over the three stocks

First, let's rework the stock prices data to compute the monthly returns for each stock.

In [14]:
stock_prices['Monthly Return'] = stock_prices.groupby(
    'Ticker')['Close Price'].pct_change()

returns_pivot = stock_prices.pivot(
    index='Date',
    columns='Ticker',
    values='Monthly Return'
).dropna()

returns_pivot.head()

Ticker,AAPL,AMZN,GE
Date,,,
2019-02,0.040315,-0.045906,0.022638
2019-03,0.097026,0.085936,-0.038499
2019-04,0.056436,0.081859,0.018018
2019-05,-0.127573,-0.078613,-0.071780
2019-06,0.130519,0.066792,0.112288


Now that we do know the returns, we can compute the covariance matrix of the three stocks over the entire period.
we know that <br> $Cov=\frac{(X-\bar{X})^T * (X-\bar{X})}{n-1}$<br>
Where $X$ is the returns matrix, $\bar{X}$ is the mean of each column (stock), and $n$ is the number of observations.

In [15]:
demeaned_returns = returns_pivot - returns_pivot.mean()
n = demeaned_returns.shape[0]
cov_matrix_manual = (demeaned_returns.T @ demeaned_returns) / (n - 1)

In [16]:
# | label: Covariance_matrix_manual
# | tbl-cap: "Covariance matrix between AAPL, AMZN and GE."
cov_matrix_manual

Ticker,AAPL,AMZN,GE
Ticker,,,
AAPL,0.013623,0.005201,0.004639
AMZN,0.005201,0.021165,0.007955
GE,0.004639,0.007955,0.726161


### Other method to compute the covariance matrix

1. using predefined pandas or numpy method
2. Manual method using the formula above (the one used here)

## Synthèse des Méthodes de Calcul de la Matrice de Covariance

| Method | Python Implementation | Pros | Cons |
| :--- | :--- | :--- | :--- |
| **1. Pandas (Built-in)** | `returns.cov()` | **Simplicity, speed, and standard** for sample covariance. Provides the **unbiased estimator** (using $N-1$). | Assumes **stationarity** (equal weight for all data points). **Slow to react** to recent market shocks. |
| **2. Manual Formula (the one we used)** | `(demeaned_returns.T @ demeaned_returns) / (N - 1)` | **Full transparency** of the calculation. Easy to implement the **Maximum Likelihood Estimator** (using $N$). | More **complex** and **less optimized** than native functions. Higher risk of implementation error. |
| **3. Decomposition ($D R D$)** | `D @ R @ D` (where $D$ is the diagonal matrix of standard deviations and $R$ is the correlation matrix) | **Guarantees consistency** between volatility and correlation inputs. Facilitates **Shrinkage** techniques. | **No performance gain** over `pandas.cov()`. Adds an intermediate calculation step. |
| **4. EWMA (Exponentially Weighted Moving Average)** | Requires specific implementation (e.g., using `pandas.ewm().cov()`). | **Better forecasting of future volatility** for risk estimation (e.g., VaR). **Reacts faster** to changes in market regime (recent data is weighted higher). | **More complex** and requires parameter tuning ($\lambda$ decay factor). The covariance matrix is **time-varying** (non-static). |